In [21]:
import pandas as pd
import numpy as np
import janitor

Données Population

In [22]:
df_population = pd.read_csv("data/population_2000_23.csv")

df_population.head(10)

,Code zone (M49),Zone,Élément,Année,Valeur
0,4,Afghanistan,Population totale,2000,20130.327
1,4,Afghanistan,Population rurale,2000,15657.474
2,4,Afghanistan,Population urbaine,2000,4436.282
3,4,Afghanistan,Population totale,2001,20284.307
4,4,Afghanistan,Population rurale,2001,16318.324
5,4,Afghanistan,Population urbaine,2001,4648.139
6,4,Afghanistan,Population totale,2002,21378.117
7,4,Afghanistan,Population rurale,2002,17086.910
8,4,Afghanistan,Population urbaine,2002,4893.013
9,4,Afghanistan,Population totale,2003,22733.049


In [23]:
df_population["Zone"].nunique()

239

In [24]:
# Multiplier la valeur par 1000
df_population["Valeur"] = df_population["Valeur"] * 1000

# Passer sur 3 colonnes (Population totale, rurale, urbaine)
df_population = df_population.pivot_table(
    index=["Code zone (M49)","Zone", "Année"],     # ce qui reste en lignes
    columns="Élément",          # ce qui devient les colonnes
    values="Valeur"             # ce qui remplit les cellules
).reset_index()

In [25]:
df_population = (
    df_population
    .rename_column("Code zone (M49)", "code_zone_m49")
    .clean_names()                     # met les noms de colonnes en snake_case
)

c:\Users\Morgi\anaconda3\Lib\site-packages\pandas_flavor\register.py:164: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.rename` instead.
  return method(self._obj, *args, **kwargs)


In [26]:
df_population.head(10)

Élément,code_zone_m49,zone,annee,population_rurale,population_totale,population_urbaine
0,4,Afghanistan,2000,15657474.0,20130327.0,4436282.0
1,4,Afghanistan,2001,16318324.0,20284307.0,4648139.0
2,4,Afghanistan,2002,17086910.0,21378117.0,4893013.0
3,4,Afghanistan,2003,17909063.0,22733049.0,5155788.0
4,4,Afghanistan,2004,18692107.0,23560654.0,5426872.0
5,4,Afghanistan,2005,19378962.0,24404567.0,5691836.0
6,4,Afghanistan,2006,19961972.0,25424094.0,5931478.0
7,4,Afghanistan,2007,20464923.0,25909852.0,6151869.0
8,4,Afghanistan,2008,20929119.0,26482622.0,6364912.0
9,4,Afghanistan,2009,21415593.0,27466100.0,6588738.0


In [27]:
df_population = df_population.sort_values(by=["zone", "annee"])

df_population["croissance_annuelle_population"] = (
    df_population.groupby("zone")["population_totale"]
    .pct_change() * 100 # .pct_change calcule automatiquement la variation en pourcentage d'une ligne par rapport à la précédente
)

C:\Users\Morgi\AppData\Local\Temp\ipykernel_25632\3763474454.py:5: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  .pct_change() * 100 # .pct_change calcule automatiquement la variation en pourcentage d'une ligne par rapport à la précédente


In [28]:
df_population.head(10)

Élément,code_zone_m49,zone,annee,population_rurale,population_totale,population_urbaine,croissance_annuelle_population
0,4,Afghanistan,2000,15657474.0,20130327.0,4436282.0,NaN
1,4,Afghanistan,2001,16318324.0,20284307.0,4648139.0,0.764916
2,4,Afghanistan,2002,17086910.0,21378117.0,4893013.0,5.392395
3,4,Afghanistan,2003,17909063.0,22733049.0,5155788.0,6.337939
4,4,Afghanistan,2004,18692107.0,23560654.0,5426872.0,3.640537
5,4,Afghanistan,2005,19378962.0,24404567.0,5691836.0,3.581874
6,4,Afghanistan,2006,19961972.0,25424094.0,5931478.0,4.177607
7,4,Afghanistan,2007,20464923.0,25909852.0,6151869.0,1.910621
8,4,Afghanistan,2008,20929119.0,26482622.0,6364912.0,2.210626
9,4,Afghanistan,2009,21415593.0,27466100.0,6588738.0,3.713673


In [29]:
df_population = df_population[df_population['annee'] >= 2019]

In [30]:
df_population["population_totale"] = pd.to_numeric(df_population["population_totale"], errors="coerce")
df_population = df_population.dropna(subset=["population_totale"])

df_population["zone"].nunique()

234

In [31]:
df_population["proportion_urbaine"] = df_population["population_urbaine"] / df_population["population_totale"] * 100

In [32]:
df_all = (
    df_population[["code_zone_m49", "zone"]]
    .drop_duplicates().reset_index(drop=True)
)

In [33]:
df_population = (
    df_population.groupby(["code_zone_m49", "zone"])[
        ["population_totale", "croissance_annuelle_population", "population_urbaine", "proportion_urbaine"]
    ]
    .mean()
    .reset_index()
)

Ajout référentiel pays et ajout premières variables au dataframe final

In [34]:
df_pays = pd.read_csv("data/ref_pays.csv")

df_pays.head(10)

,Code Pays,Pays,Code M49,Code ISO2,Code ISO3
0,2,Afghanistan,4,AF,AFG
1,5100,Afrique,2,F5100,X06
2,5104,Afrique australe,18,F5104,F5104
3,5102,Afrique centrale,17,F5102,F5102
4,429,Afrique du Nord (Soudan Excl),746,F429,F429
5,202,Afrique du Sud,710,ZA,ZAF
6,5105,Afrique occidentale,11,F5105,F5105
7,5101,Afrique orientale,14,F5101,F5101
8,5103,Afrique septentrionale,15,F5103,F5103
9,420,Afrique subsaharienne,202,F420,F420


In [35]:
df_pays = df_pays.clean_names()

In [36]:
df_all = df_all.merge(
    df_pays[["pays","code_iso2", "code_iso3"]],
    left_on="zone",
    right_on="pays",
    how="left"
).drop(columns=["zone"])

In [37]:
df_all.head()

,code_zone_m49,pays,code_iso2,code_iso3
0,4,Afghanistan,AF,AFG
1,710,Afrique du Sud,ZA,ZAF
2,8,Albanie,AL,ALB
3,12,Algérie,DZ,DZA
4,276,Allemagne,DE,DEU


In [38]:
df_all = df_all.merge(
    df_population[["zone","population_totale","population_urbaine","proportion_urbaine","croissance_annuelle_population"]],
    left_on="pays",
    right_on="zone",
    how="left"
).drop(columns=["zone"])

In [39]:
df_all["proportion_urbaine"] = df_all["proportion_urbaine"].clip(upper=100).round(2)

df_all["croissance_annuelle_population"] = df_all["croissance_annuelle_population"].clip(upper=100).round(2)

Distances

In [41]:
df_distances = pd.read_csv("data/distances.csv")

In [42]:
df_distances = df_distances[df_distances["iso_d"] == "FRA"]

In [43]:
df_distances.head()

,iso_o,iso_d,contig,dist
67,ABW,FRA,0,"7685,884"
291,AFG,FRA,0,"5590,381"
515,AGO,FRA,0,"6510,322"
739,AIA,FRA,0,"6710,57"
963,ALB,FRA,0,"1603,534"


In [44]:
df_distances["iso_o"].nunique()

224

In [45]:
df_all = df_all.merge(
    df_distances[["iso_o","contig","dist"]],
    left_on="code_iso3",
    right_on="iso_o",
    how="left"
).drop(columns=["iso_o"])

In [46]:
df_all.head()

,code_zone_m49,pays,code_iso2,code_iso3,population_totale,population_urbaine,proportion_urbaine,croissance_annuelle_population,contig,dist
0,4,Afghanistan,AF,AFG,39791823.0,10252456.8,25.75,2.44,0.0,"5590,381"
1,710,Afrique du Sud,ZA,ZAF,61448732.6,40271816.6,65.53,1.52,0.0,"9353,552"
2,8,Albanie,AL,ALB,2849172.4,1852955.8,65.05,-0.58,0.0,"1603,534"
3,12,Algérie,DZ,DZA,44747868.8,32636191.4,72.92,1.67,0.0,"1340,39"
4,276,Allemagne,DE,DEU,83903887.0,64018680.0,76.30,0.28,1.0,"439,8984"


Indicateurs politiques

In [48]:
df_stabilite = pd.read_csv("data/political_stability_perc_2023.csv")

In [49]:
df_stabilite.head()

,Country Name,Country Code,Series Name,Series Code,2023
0,Afghanistan,AFG,Political Stability and Absence of Violence/Te...,PV.PER.RNK,1.42180097103119
1,Albania,ALB,Political Stability and Absence of Violence/Te...,PV.PER.RNK,51.6587677001953
2,Algeria,DZA,Political Stability and Absence of Violence/Te...,PV.PER.RNK,23.222749710083
3,American Samoa,ASM,Political Stability and Absence of Violence/Te...,PV.PER.RNK,91.9431304931641
4,Andorra,AND,Political Stability and Absence of Violence/Te...,PV.PER.RNK,98.5782012939453


In [50]:
df_stabilite["Country Code"].nunique()

214

In [51]:
df_stabilite = (
    df_stabilite
    .rename_column("2023", "political_stability")
    .clean_names()
)

c:\Users\Morgi\anaconda3\Lib\site-packages\pandas_flavor\register.py:164: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.rename` instead.
  return method(self._obj, *args, **kwargs)


In [52]:
df_stabilite['political_stability'] = pd.to_numeric(
    df_stabilite['political_stability'], errors='coerce'
).round(2)

In [53]:
df_all = df_all.merge(
    df_stabilite[["country_code", "political_stability"]],
    left_on="code_iso3",
    right_on="country_code",
    how="left"
).drop(columns=["country_code"])

In [55]:
df_regulatory = pd.read_csv("data/regulatory_quality_perc_2023.csv")

In [56]:
df_regulatory.head()

,Country Name,Country Code,Series Name,Series Code,2023 [YR2023]
0,Afghanistan,AFG,Regulatory Quality: Percentile Rank,RQ.PER.RNK,9.43396186828613
1,Albania,ALB,Regulatory Quality: Percentile Rank,RQ.PER.RNK,58.4905662536621
2,Algeria,DZA,Regulatory Quality: Percentile Rank,RQ.PER.RNK,16.0377349853516
3,American Samoa,ASM,Regulatory Quality: Percentile Rank,RQ.PER.RNK,68.3962249755859
4,Andorra,AND,Regulatory Quality: Percentile Rank,RQ.PER.RNK,89.6226425170898


In [57]:
df_regulatory = (
    df_regulatory
    .rename_column("2023 [YR2023]", "regulatory_quality")
    .clean_names()
)

c:\Users\Morgi\anaconda3\Lib\site-packages\pandas_flavor\register.py:164: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.rename` instead.
  return method(self._obj, *args, **kwargs)


In [58]:
df_regulatory['regulatory_quality'] = pd.to_numeric(
    df_regulatory['regulatory_quality'], errors='coerce'
).round(2)

In [59]:
df_all = df_all.merge(
    df_regulatory[["country_code", "regulatory_quality"]],
    left_on="code_iso3",
    right_on="country_code",
    how="left"
).drop(columns=["country_code"])

Economie

In [61]:
df_PIB = pd.read_csv("data/PIB_2010_24.csv")

In [62]:
df_PIB.head()

,Code zone (M49),Zone,Élément,Produit,Année,Unité,Valeur
0,4,Afghanistan,Valeur US$,Produit Intérieur Brut,2010,Millions d’USD,15144.569659
1,4,Afghanistan,Valeur US$,Produit Intérieur Brut,2011,Millions d’USD,17923.100250
2,4,Afghanistan,Valeur US$,Produit Intérieur Brut,2012,Millions d’USD,19794.405855
3,4,Afghanistan,Valeur US$,Produit Intérieur Brut,2013,Millions d’USD,19904.409340
4,4,Afghanistan,Valeur US$,Produit Intérieur Brut,2014,Millions d’USD,19500.460924


In [63]:
df_PIB = (
    df_PIB
    .rename_column("Valeur", "produit_interieur_brut_usd")
    .clean_names()
)

c:\Users\Morgi\anaconda3\Lib\site-packages\pandas_flavor\register.py:164: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.rename` instead.
  return method(self._obj, *args, **kwargs)


In [64]:
df_PIB["produit_interieur_brut_usd"] = df_PIB["produit_interieur_brut_usd"] * 1000000

In [65]:
df_PIB.head()

,code_zone_m49_,zone,element,produit,annee,unite,produit_interieur_brut_usd
0,4,Afghanistan,Valeur US$,Produit Intérieur Brut,2010,Millions d’USD,1.514457e+10
1,4,Afghanistan,Valeur US$,Produit Intérieur Brut,2011,Millions d’USD,1.792310e+10
2,4,Afghanistan,Valeur US$,Produit Intérieur Brut,2012,Millions d’USD,1.979441e+10
3,4,Afghanistan,Valeur US$,Produit Intérieur Brut,2013,Millions d’USD,1.990441e+10
4,4,Afghanistan,Valeur US$,Produit Intérieur Brut,2014,Millions d’USD,1.950046e+10


In [66]:
df_PIB = df_PIB.sort_values(by=["zone", "annee"])

df_PIB["croissance_annuelle_pib"] = (
    df_PIB.groupby("zone")["produit_interieur_brut_usd"]
    .pct_change() * 100 # .pct_change calcule automatiquement la variation en pourcentage d'une ligne par rapport à la précédente
)

In [67]:
df_PIB = df_PIB[df_PIB['annee'] >= 2020]

In [68]:
df_PIB = (
    df_PIB.groupby(["code_zone_m49_", "zone"])[
        ["produit_interieur_brut_usd", "croissance_annuelle_pib"]
    ]
    .mean()
    .reset_index()
)

In [69]:
df_PIB["croissance_annuelle_pib"] = df_PIB["croissance_annuelle_pib"].clip(upper=100).round(2)

In [70]:
df_all = df_all.merge(
    df_PIB[["code_zone_m49_", "produit_interieur_brut_usd", "croissance_annuelle_pib"]],
    left_on="code_zone_m49",
    right_on="code_zone_m49_",
    how="left"
).drop(columns=["code_zone_m49_"])

In [71]:
df_all["pib_par_habitant"] = df_all["produit_interieur_brut_usd"] / df_all["population_totale"]

df_all["pib_par_habitant"] = df_all["pib_par_habitant"].round(2)

In [73]:
df_taux_change = pd.read_csv("data/taux_change_LCU_USD_2000_24.csv")

In [74]:
df_taux_change.head()

,Code zone (M49),Zone,Code de devise ISO (FAO),Devise,Code Élément,Élément,Code année,Année,Code Mois,Mois,Valeur
0,4,Afghanistan,AFA,Afghani,LCU,Unités de monnaie locale par USD,2000,2000,7021,Valeur annuel,47357.574730
1,4,Afghanistan,AFA,Afghani,LCU,Unités de monnaie locale par USD,2001,2001,7021,Valeur annuel,47500.014520
2,4,Afghanistan,AFA,Afghani,LCU,Unités de monnaie locale par USD,2002,2002,7021,Valeur annuel,47263.000000
3,4,Afghanistan,AFN,Afghani,LCU,Unités de monnaie locale par USD,2003,2003,7021,Valeur annuel,48.762754
4,4,Afghanistan,AFN,Afghani,LCU,Unités de monnaie locale par USD,2004,2004,7021,Valeur annuel,47.845312


In [75]:
df_taux_change = (
    df_taux_change
    .rename_column("Valeur", "taux_change_usd_lcu")
    .clean_names()                     # met les noms de colonnes en snake_case
)

c:\Users\Morgi\anaconda3\Lib\site-packages\pandas_flavor\register.py:164: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.rename` instead.
  return method(self._obj, *args, **kwargs)


In [76]:
df_taux_change = df_taux_change.sort_values(by=["zone", "annee"])

df_taux_change["variation_taux_change"] = (
    df_taux_change.groupby("zone")["taux_change_usd_lcu"]
    .pct_change() * 100 # .pct_change calcule automatiquement la variation en pourcentage d'une ligne par rapport à la précédente
)

In [77]:
# variation logarithmique (utilisée en finance)

def log_return(x):
    return (np.log(x) - np.log(x.shift(1))) * 100

df_taux_change["variation_log_taux_change"] = df_taux_change.groupby("zone")["taux_change_usd_lcu"].apply(log_return).reset_index(level=0, drop=True)

# à corriger

In [78]:
df_taux_change

,code_zone_m49_,zone,code_de_devise_iso_fao_,devise,code_element,element,code_annee,annee,code_mois,mois,taux_change_usd_lcu,variation_taux_change,variation_log_taux_change
0,4,Afghanistan,AFA,Afghani,LCU,Unités de monnaie locale par USD,2000,2000,7021,Valeur annuel,47357.574730,NaN,NaN
1,4,Afghanistan,AFA,Afghani,LCU,Unités de monnaie locale par USD,2001,2001,7021,Valeur annuel,47500.014520,0.300775,0.300324
2,4,Afghanistan,AFA,Afghani,LCU,Unités de monnaie locale par USD,2002,2002,7021,Valeur annuel,47263.000000,-0.498978,-0.500227
3,4,Afghanistan,AFN,Afghani,LCU,Unités de monnaie locale par USD,2003,2003,7021,Valeur annuel,48.762754,-99.896827,-687.651625
4,4,Afghanistan,AFN,Afghani,LCU,Unités de monnaie locale par USD,2004,2004,7021,Valeur annuel,47.845312,-1.881438,-1.899362
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2296,248,Îles d'Åland,EUR,Euro,LCU,Unités de monnaie locale par USD,2020,2020,7021,Valeur annuel,0.875506,-1.989291,-2.009343
2297,248,Îles d'Åland,EUR,Euro,LCU,Unités de monnaie locale par USD,2021,2021,7021,Valeur annuel,0.845494,-3.427988,-3.488122
2298,248,Îles d'Åland,EUR,Euro,LCU,Unités de monnaie locale par USD,2022,2022,7021,Valeur annuel,0.949624,12.315829,11.614462
2299,248,Îles d'Åland,EUR,Euro,LCU,Unités de monnaie locale par USD,2023,2023,7021,Valeur annuel,0.924840,-2.609896,-2.644559


In [79]:
df_taux_change = df_taux_change[df_taux_change['annee'] >= 2019]

In [80]:
df_taux_change["flag_euro"] = (df_taux_change["code_de_devise_iso_fao_"] == "EUR").astype(int) # renvoie TRUE si vrai puis convertit en 1 ou 0 en fonction de TRUE ou FALSE

C:\Users\Morgi\AppData\Local\Temp\ipykernel_25632\1733794427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_taux_change["flag_euro"] = (df_taux_change["code_de_devise_iso_fao_"] == "EUR").astype(int) # renvoie TRUE si vrai puis convertit en 1 ou 0 en fonction de TRUE ou FALSE


In [81]:
df_taux_change = (
    df_taux_change.groupby(["code_zone_m49_", "zone", "flag_euro"])[
        ["variation_log_taux_change"]
    ]
    .mean()
    .reset_index()
)

In [82]:
df_all = df_all.merge(
    df_taux_change[["code_zone_m49_", "variation_log_taux_change", "flag_euro"]],
    left_on="code_zone_m49",
    right_on="code_zone_m49_",
    how="left"
).drop(columns=["code_zone_m49_"])

Poulet : disponibilité, production et importation

In [84]:
df_dispo_volaille = pd.read_csv("data/dispo_volaille_2010_22.csv")

df_production = pd.read_csv("data/production_poulet_2020_23.csv")

df_import = pd.read_csv("data/quantité_import_2000_23.csv")

df_prix_import = pd.read_csv("data/prix_import_2000_23.csv")

df_prix_production = pd.read_csv("data/prix_production_2020_23.csv")

In [85]:
df_dispo_volaille.head()

,Code zone (M49),Zone,Élément,Produit,Année,Unité,Valeur
0,4,Afghanistan,Disponibilité intérieure,Viande de Volailles,2010,1000 t,65
1,4,Afghanistan,Disponibilité intérieure,Viande de Volailles,2011,1000 t,56
2,4,Afghanistan,Disponibilité intérieure,Viande de Volailles,2012,1000 t,62
3,4,Afghanistan,Disponibilité intérieure,Viande de Volailles,2013,1000 t,67
4,4,Afghanistan,Disponibilité intérieure,Viande de Volailles,2014,1000 t,69


In [86]:
df_production.head()

,Code zone (M49),Zone,Élément,Produit,Année,Unité,Valeur
0,4,Afghanistan,Production,"Viande, poulet, fraîche ou réfrigérée",2020,tonnes,28512.87
1,4,Afghanistan,Production,"Viande, poulet, fraîche ou réfrigérée",2021,tonnes,28210.69
2,4,Afghanistan,Production,"Viande, poulet, fraîche ou réfrigérée",2022,tonnes,28503.14
3,4,Afghanistan,Production,"Viande, poulet, fraîche ou réfrigérée",2023,tonnes,28796.08
4,710,Afrique du Sud,Production,"Viande, poulet, fraîche ou réfrigérée",2020,tonnes,1887000.00


In [87]:
df_import.head()

,Code zone (M49),Zone,Élément,Produit,Année,Unité,Valeur
0,4,Afghanistan,Importations - quantité,"Viande, poulet, fraîche ou réfrigérée",2000,tonnes,25.0
1,4,Afghanistan,Importations - quantité,"Viande, poulet, fraîche ou réfrigérée",2001,tonnes,45.0
2,4,Afghanistan,Importations - quantité,"Viande, poulet, fraîche ou réfrigérée",2002,tonnes,1068.0
3,4,Afghanistan,Importations - quantité,"Viande, poulet, fraîche ou réfrigérée",2003,tonnes,316.0
4,4,Afghanistan,Importations - quantité,"Viande, poulet, fraîche ou réfrigérée",2004,tonnes,4933.0


In [88]:
df_prix_import.head()

,Code zone (M49),Zone,Élément,Produit,Année,Unité,Valeur
0,4,Afghanistan,Importations - valeur,"Viande, poulet, fraîche ou réfrigérée",2000,1000 USD,25
1,4,Afghanistan,Importations - valeur,"Viande, poulet, fraîche ou réfrigérée",2001,1000 USD,45
2,4,Afghanistan,Importations - valeur,"Viande, poulet, fraîche ou réfrigérée",2002,1000 USD,1262
3,4,Afghanistan,Importations - valeur,"Viande, poulet, fraîche ou réfrigérée",2003,1000 USD,458
4,4,Afghanistan,Importations - valeur,"Viande, poulet, fraîche ou réfrigérée",2004,1000 USD,4183


In [89]:
df_prix_production.head()

,Code zone (M49),Zone,Élément,Produit,Année,Unité,Valeur
0,710,Afrique du Sud,Prix à la Production (USD/tonne),"Viande, poulet, fraîche ou réfrigérée",2020,USD,1612.7
1,710,Afrique du Sud,Prix à la Production (USD/tonne),"Viande, poulet, fraîche ou réfrigérée",2021,USD,1791.2
2,710,Afrique du Sud,Prix à la Production (USD/tonne),"Viande, poulet, fraîche ou réfrigérée",2022,USD,1847.9
3,710,Afrique du Sud,Prix à la Production (USD/tonne),"Viande, poulet, fraîche ou réfrigérée",2023,USD,1914.6
4,8,Albanie,Prix à la Production (USD/tonne),"Viande, poulet, fraîche ou réfrigérée",2020,USD,4711.4


In [90]:
df_dispo_volaille = (
    df_dispo_volaille
    .rename_column("Valeur", "dispo_int_volaille")
    .clean_names()
)
df_production = (
    df_production
    .clean_names()
    .rename_column("valeur", "prod_poulet_t")
)
df_import = (
    df_import
    .rename_column("Valeur", "import_poulet_t")
    .clean_names()
)
df_prix_import = (
    df_prix_import
    .rename_column("Valeur", "prix_import_poulet_usd_total")
    .clean_names()
)
df_prix_production = (
    df_prix_production
    .rename_column("Valeur", "prix_production_poulet_usd_tonne")
    .clean_names()
)

c:\Users\Morgi\anaconda3\Lib\site-packages\pandas_flavor\register.py:164: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.rename` instead.
  return method(self._obj, *args, **kwargs)
c:\Users\Morgi\anaconda3\Lib\site-packages\pandas_flavor\register.py:164: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.rename` instead.
  return method(self._obj, *args, **kwargs)
c:\Users\Morgi\anaconda3\Lib\site-packages\pandas_flavor\register.py:164: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.rename` instead.
  return method(self._obj, *args, **kwargs)
c:\Users\Morgi\anaconda3\Lib\site-packages\pandas_flavor\register.py:164: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.rename` instead.
  return method(self._obj, *args, **kwargs)


In [91]:
df_dispo_volaille["dispo_int_volaille"] = df_dispo_volaille["dispo_int_volaille"] * 1000

df_prix_import["prix_import_poulet_usd_total"] = df_prix_import["prix_import_poulet_usd_total"] * 1000

In [92]:
def keep_last_year(df):
    return (
        df.sort_values(["code_zone_m49_", "annee"], ascending=[True, False])
          .drop_duplicates(subset="code_zone_m49_", keep="first")
    )

In [93]:
df_production = keep_last_year(df_production)
df_import = keep_last_year(df_import)
df_prix_import = keep_last_year(df_prix_import)
df_dispo_volaille = keep_last_year(df_dispo_volaille)
df_prix_production = keep_last_year(df_prix_production)

In [94]:
def to_merge(df_all, df, new_value):
    df_all = df_all.merge(
        df[["code_zone_m49_", new_value]],
        left_on="code_zone_m49",
        right_on="code_zone_m49_",
        how="left"
    ).drop(columns=["code_zone_m49_"])
    return df_all

In [95]:
df_all = to_merge(df_all, df_production, "prod_poulet_t")

In [96]:
df_all = to_merge(df_all, df_import, "import_poulet_t")

In [97]:
df_all = to_merge(df_all, df_prix_import, "prix_import_poulet_usd_total")

In [98]:
df_all = to_merge(df_all, df_prix_production, "prix_production_poulet_usd_tonne")

In [99]:
df_all['prix_import_poulet_usd_tonne'] = df_all['prix_import_poulet_usd_total'] / df_all['import_poulet_t']

In [100]:
df_all = to_merge(df_all, df_dispo_volaille, "dispo_int_volaille")

In [101]:
df_all['import_poulet_t_hab'] = df_all['import_poulet_t'] / df_all['population_totale']

df_all['dispo_int_volaille_hab'] = df_all['dispo_int_volaille'] / df_all['population_totale']

df_all['prod_poulet_t_hab'] = df_all['prod_poulet_t'] / df_all['population_totale']

Exploration et nettoyage

In [102]:
df_all['prix_import_poulet_usd_tonne'].max()

inf

In [103]:
df_all['prix_import_poulet_usd_tonne'].replace([np.inf, -np.inf], np.nan, inplace=True)

C:\Users\Morgi\AppData\Local\Temp\ipykernel_25632\2802079291.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_all['prix_import_poulet_usd_tonne'].replace([np.inf, -np.inf], np.nan, inplace=True)


In [104]:
df_all['prix_import_poulet_usd_tonne'].max()

8695.652173913044

In [105]:
df_all['population_totale'].max()

1456459482.8

In [106]:
df_all['population_totale'].min()

1808.6

In [107]:
df_all.loc[
    df_all.isna().sum(axis=1) >= 9
]

,code_zone_m49,pays,code_iso2,code_iso3,population_totale,population_urbaine,proportion_urbaine,croissance_annuelle_population,contig,dist,...,flag_euro,prod_poulet_t,import_poulet_t,prix_import_poulet_usd_total,prix_production_poulet_usd_tonne,prix_import_poulet_usd_tonne,dispo_int_volaille,import_poulet_t_hab,dispo_int_volaille_hab,prod_poulet_t_hab
5,20,Andorre,AD,AND,7.855580e+04,67912.4,86.48,1.47,1.0,"711,5485",...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,660,Anguilla,AI,AIA,1.454620e+04,15362.8,100.00,-0.41,0.0,"6710,57",...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,533,Aruba,AW,ABW,1.075510e+05,46861.4,43.57,0.28,0.0,"7685,884",...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,654,"Ascension, Sainte-Hélène et Tristan da Cunha",SH,SHN,5.382000e+03,1669.6,31.03,-0.71,0.0,"7254,018",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,60,Bermudes,BM,BMU,6.451680e+04,60510.8,93.79,0.27,0.0,"5764,324",...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,535,"Bonaire, Saint-Eustache et Saba",BQ,BES,2.758520e+04,19852.2,72.13,3.52,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,159,Chine,F351,F351,1.456459e+09,919369954.2,63.12,0.05,NaN,NaN,...,NaN,15476826.35,1788322.86,5.108129e+09,NaN,2856.379636,25364000.0,0.001228,0.017415,0.010626
54,531,Curaçao,CW,CUW,1.861296e+05,146454.8,78.69,0.44,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,292,Gibraltar,GI,GIB,3.692100e+04,35112.6,95.17,1.99,0.0,"1549,175",...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,304,Groenland,GL,GRL,5.602340e+04,49773.6,88.84,-0.05,0.0,"3584,681",...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
df_all.loc[
    df_all["population_totale"] < 50000
]

,code_zone_m49,pays,code_iso2,code_iso3,population_totale,population_urbaine,proportion_urbaine,croissance_annuelle_population,contig,dist,...,flag_euro,prod_poulet_t,import_poulet_t,prix_import_poulet_usd_total,prix_production_poulet_usd_tonne,prix_import_poulet_usd_tonne,dispo_int_volaille,import_poulet_t_hab,dispo_int_volaille_hab,prod_poulet_t_hab
7,660,Anguilla,AI,AIA,14546.2,15362.8,100.00,-0.41,0.0,"6710,57",...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,654,"Ascension, Sainte-Hélène et Tristan da Cunha",SH,SHN,5382.0,1669.6,31.03,-0.71,0.0,"7254,018",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,535,"Bonaire, Saint-Eustache et Saba",BQ,BES,27585.2,19852.2,72.13,3.52,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,292,Gibraltar,GI,GIB,36921.0,35112.6,95.17,1.99,0.0,"1549,175",...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,438,Liechtenstein,LI,LIE,39033.2,5632.4,14.43,0.76,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,492,Monaco,MC,MCO,38514.6,39493.6,100.00,0.42,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,500,Montserrat,MS,MSR,4480.0,482.0,10.76,-0.66,0.0,"6771,885",...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,520,Nauru,NR,NRU,11723.0,11240.4,95.89,0.68,0.0,"14455,09",...,0.0,4.02,587.56,1474000.0,NaN,2508.679965,1000.0,0.050120,0.085302,0.000343
135,570,Nioué,NU,NIU,1808.6,764.6,42.27,0.35,0.0,"16639,5",...,NaN,25.51,75.50,353000.0,NaN,4675.496689,NaN,0.041745,NaN,0.014105
144,585,Palaos,PW,PLW,17771.8,18481.0,100.00,-0.10,0.0,"12240,69",...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
df_all.loc[
    df_all["pays"].str.startswith("Chine")
]

# Chine a beaucoup de valeurs nulles, et celles qui ne le sont pas sont la somme des autres donc on peut retirer ce pays

,code_zone_m49,pays,code_iso2,code_iso3,population_totale,population_urbaine,proportion_urbaine,croissance_annuelle_population,contig,dist,...,flag_euro,prod_poulet_t,import_poulet_t,prix_import_poulet_usd_total,prix_production_poulet_usd_tonne,prix_import_poulet_usd_tonne,dispo_int_volaille,import_poulet_t_hab,dispo_int_volaille_hab,prod_poulet_t_hab
41,159,Chine,F351,F351,1.456459e+09,919369954.2,63.12,0.05,NaN,NaN,...,NaN,15476826.35,1788322.86,5.108129e+09,NaN,2856.379636,25364000.0,0.001228,0.017415,0.010626
42,344,Chine - RAS de Hong-Kong,HK,HKG,7.473860e+06,7592934.8,100.00,-0.08,0.0,"9639,476",...,0.0,23641.87,236627.79,5.773650e+08,7196.0,2439.971231,356000.0,0.031661,0.047633,0.003163
43,446,Chine - RAS de Macao,MO,MAC,6.933422e+05,661907.4,95.47,1.62,0.0,"9601,923",...,0.0,3184.49,26595.59,7.278900e+07,NaN,2736.882318,34000.0,0.038359,0.049038,0.004593
44,158,"Chine, Taiwan Province de",TW,TWN,2.352664e+07,18931297.8,80.47,-0.29,0.0,"9834,232",...,0.0,650000.00,239529.11,3.267530e+08,NaN,1364.147347,977000.0,0.010181,0.041527,0.027628
45,156,"Chine, continentale",CN,CHN,1.424766e+09,892183814.2,62.62,0.05,0.0,"8225,232",...,0.0,14800000.00,1285570.36,4.131223e+09,3408.3,3213.533174,23996000.0,0.000902,0.016842,0.010388


In [110]:
pays_exclus = ["France", 
               "Ascension, Sainte-Hélène et Tristan da Cunha", 
               "Chine", 
               "Bonaire, Saint-Eustache et Saba", 
               "Nioué", 
               "Île de Man", 
               "Montserrat", 
               "Îles Falkland (Malvinas)", 
               "Saint-Martin (partie française)",
               "Tokélaou",
               "Saint-Pierre-et-Miquelon",
               "Saint-Barthélemy",
               "Îles Mariannes du Nord",
               "Îles Vierges américaines",
               "Îles Wallis-et-Futuna",
               "Samoa américaines",
               "Guam",
               "Sahara occidental",
               "Mayotte",
               "Gibraltar",
               "Îles Vierges britanniques",
               "Îles Turques-et-Caïques",
               "Îles Caïmanes",
               "Curaçao",
               "Sint Maarten (partie néerlandaise)",
               "Saint-Marin",
               "Palaos",
               "Monaco",
               "Liechtenstein",
               "Anguilla",
               "Groenland",
               "Bermudes",
               "Aruba",
               "Guyane française",
               "Réunion",
               "Guadeloupe",
               "Andorre",
               "Îles Féroé",
               "Martinique"
               ]

df_final = df_all[~df_all["pays"].isin(pays_exclus)]

In [111]:
df_final["pays"].nunique()

195

In [112]:
df_final = df_final.drop(columns=["code_zone_m49", "code_iso2"])

In [113]:
df_final.isna().mean()

pays                                0.000000
code_iso3                           0.000000
population_totale                   0.000000
population_urbaine                  0.000000
proportion_urbaine                  0.000000
croissance_annuelle_population      0.000000
contig                              0.035714
dist                                0.035714
political_stability                 0.010204
regulatory_quality                  0.010204
produit_interieur_brut_usd          0.005102
croissance_annuelle_pib             0.005102
pib_par_habitant                    0.005102
variation_log_taux_change           0.000000
flag_euro                           0.000000
prod_poulet_t                       0.020408
import_poulet_t                     0.015306
prix_import_poulet_usd_total        0.015306
prix_production_poulet_usd_tonne    0.295918
prix_import_poulet_usd_tonne        0.020408
dispo_int_volaille                  0.035714
import_poulet_t_hab                 0.015306
dispo_int_

In [115]:
df_final.to_csv("data/df_all.csv", sep=";", index=False, encoding="utf-8")